In [1]:
import pandas as pd
import pickle
import sklearn
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# mainpath = 'drive/MyDrive/+Renzoe Box/Administrative/Team Resources/Team Folders/Quhan Peng/NLP process/'
# data = pd.read_csv(mainpath + 'testdata_after_sample800.csv')

data = pd.read_csv('testdata_after_sample800.csv')

In [3]:
# Delete products with keyword 'loose powder'
loose_powder = data[data['body (html)'].str.contains('loose powder')]
nlpdata = data[~data.isin(loose_powder)]

In [4]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# set english stopwords
stop_words = set(stopwords.words('english'))

# Apply basic NLP for one piece of text
def clean_text(text):
    
# Remove \n\w\s, all lower case
    text = re.sub(r'[^\w\s]', ' ', str(text).lower().strip())
    text = text.replace('\n', ' ')
    lst_text = text.split()
    
    # Remove stopwords
    lst_text = [word for word in lst_text if word not in stop_words]
    
    # Stemming (remove -ing, -ly, ...)
    ps = nltk.stem.porter.PorterStemmer()
    lst_text = [ps.stem(word) for word in lst_text]
    
    # Lemmatisation (convert the word into root word)
    lem = nltk.stem.wordnet.WordNetLemmatizer()
    lst_text = [lem.lemmatize(word) for word in lst_text]
    text = " ".join(lst_text)
    
    return text

# Apply clean_text function the 'text' column for each row in df 
def NLPCleaning(df):
    data = df.copy()
    data["text"] = data["text"].apply(clean_text)
    return data

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
# Delete na and duplicate
nlpdata = nlpdata.dropna(subset=['body (html)'])
### Duplicate need to be adjusted based on new web scrapper results
nlpdata = nlpdata.drop_duplicates()

nlpdata['text'] = nlpdata['handle'] + " " + nlpdata['title'] + " " + nlpdata['body (html)']
# Apply nlp
nlpdata = NLPCleaning(nlpdata)

In [6]:
# transfer mlb result from (a, b, c) to "a, b, c"

def mlb2string(mlbresult):
    theresult = []
    for i in range(len(mlbresult)):
        t = ""
        for j in range(len(mlbresult[i])):
            t = t + mlbresult[i][j] + ", "
        t = t.rstrip(", ")
        theresult.append(t)
    
    return theresult

In [7]:
#product type
product_type_vectorizer = pickle.load(open('product_type_vectorizer.sav', 'rb'))
product_type_finalized_model = pickle.load(open('product_type_finalized_model.sav', 'rb'))

#fit trained product type TfidfVectorizer for 'text'
vectorised_product_type_documents = product_type_vectorizer.transform(nlpdata['text'])

#fit product type final model
product_type_predict = product_type_finalized_model.predict(vectorised_product_type_documents)
nlpdata['product type'] = product_type_predict

In [8]:
#product consistency
product_consistency_vectorizer = pickle.load(open('product_consistency_vectorizer.sav', 'rb'))
product_consistency_finalized_model = pickle.load(open('product_consistency_finalized_model.sav', 'rb'))

#fit trained product type TfidfVectorizer for 'text'
vectorised_product_consistency_documents = product_consistency_vectorizer.transform(nlpdata['text'])

#fit product type final model
product_consistency_predict = product_consistency_finalized_model.predict(vectorised_product_consistency_documents)
nlpdata['product consistency'] = product_consistency_predict

In [9]:
# pod nonpod
nlpdata['pod nonpod'] = ""

for i in range(len(nlpdata)):
    if((nlpdata['product consistency'].iloc[i] == 'pencil') | (nlpdata['product consistency'].iloc[i] == 'liquid')):
        nlpdata['pod nonpod'].iloc[i] = 'nonpod'
  
    if((nlpdata['product consistency'].iloc[i] == 'pressed powder') | (nlpdata['product consistency'].iloc[i] == 'cake')):
        nlpdata['pod nonpod'].iloc[i] = 'pod'
  
    elif(nlpdata['product consistency'].iloc[i] == 'cream'):
        if(nlpdata['product type'].iloc[i] == 'concealer'):
            nlpdata['pod nonpod'].iloc[i] = 'nonpod'
        else:
            nlpdata['pod nonpod'].iloc[i] = 'pod'
    
    elif(nlpdata['product consistency'].iloc[i] == 'stick'):
        if((nlpdata['product type'].iloc[i] == 'eyeliner') | (nlpdata['product type'].iloc[i] == 'lipstick')):
            nlpdata['pod nonpod'].iloc[i] = 'nonpod'

        else:
            nlpdata['pod nonpod'].iloc[i] = 'pod'


D:\Python\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
# product preference
# import models

product_preference_vectorizer = pickle.load(open('product_preference_vectorizer.sav', 'rb'))
product_preference_mlb = pickle.load(open('product_preference_mlb.sav', 'rb'))
product_preference_model = pickle.load(open('product_preference_finalized_model.sav', 'rb'))

# fit trained TfidfVectorizer for 'text'
vectorised_product_preference_documents = product_preference_vectorizer.transform(nlpdata['text'])
# fit product type final model
product_preference_predict = product_preference_model.predict(vectorised_product_preference_documents)
product_preference_predict = product_preference_mlb.inverse_transform(product_preference_predict)
nlpdata['product preference'] = mlb2string(product_preference_predict)

In [11]:
# skin concern and converge type
# Only apply for product type =  fundation | concealer
sc_ct_data = nlpdata[(nlpdata['product type'] == 'foundation') | (nlpdata['product type'] == 'concealer')]

In [12]:
#coverage type
#import models
coverage_type_finalized_model = pickle.load(open('coverage_type_finalized_model.sav', 'rb'))
coverage_type_vectorizer = pickle.load(open('coverage_type_vectorizer.sav', 'rb'))

# fit trained TfidfVectorizer for 'text'
vectorised_coverage_type_documents = coverage_type_vectorizer.transform(sc_ct_data['text'])

# fit product type final model
coverage_type_predict = coverage_type_finalized_model.predict(vectorised_coverage_type_documents)
sc_ct_data['coverage type'] = coverage_type_predict

C:\Users\DELL\AppData\Local\Temp/ipykernel_4252/4270425432.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sc_ct_data['coverage type'] = coverage_type_predict


In [13]:
#skin concern
#import models
skin_concern_vectorizer = pickle.load(open('skin_concern_vectorizer.sav', 'rb'))
skin_concern_mlb = pickle.load(open('skin_concern_mlb.sav', 'rb'))
skin_concern_model = pickle.load(open('skin_concern_finalized_model.sav', 'rb'))

# fit trained TfidfVectorizer for 'text'
vectorised_skin_concern_documents = skin_concern_vectorizer.transform(sc_ct_data['text'])

# fit product type final model
skin_concern_predict = skin_concern_model.predict(vectorised_skin_concern_documents)
skin_concern_predict = skin_concern_mlb.inverse_transform(skin_concern_predict)
sc_ct_data['skin concern'] = mlb2string(skin_concern_predict)

C:\Users\DELL\AppData\Local\Temp/ipykernel_4252/2039948381.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sc_ct_data['skin concern'] = mlb2string(skin_concern_predict)


In [14]:
nlpdata['skin concern'] = ""
nlpdata['coverage type'] = ""
nlpdata.update(sc_ct_data)

In [15]:
# finish
# everything except setting powder, eyeliner, mascara, brow powder/gel
finish_data = nlpdata[(nlpdata['product type'] != 'setting powder') & (nlpdata['product type'] != 'eyeliner') 
            & (nlpdata['product type'] != 'mascara') & (nlpdata['product type'] != 'brow')]

In [16]:
#finish
finish_finalized_model = pickle.load(open('finish_finalized_model.sav', 'rb'))
finish_vectorizer = pickle.load(open('finish_vectorizer.sav', 'rb'))

# fit trained TfidfVectorizer for 'text'
vectorised_finish_documents = finish_vectorizer.transform(finish_data['text'])

# fit product type final model
finish_predict = finish_finalized_model.predict(vectorised_finish_documents)

finish_data['finish'] = finish_predict
nlpdata['finish'] = ""
nlpdata.update(finish_data)

C:\Users\DELL\AppData\Local\Temp/ipykernel_4252/2410460293.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finish_data['finish'] = finish_predict


In [17]:
nlpdata = nlpdata.drop(columns = 'text')

In [18]:
#output
#nlpdata.to_csv('nlpresult.csv')
#from google.colab import files
#files.download("nlpresult.csv")

In [19]:
# update nlpdata['tags'] 
nlpdata['tags'] = nlpdata['pod nonpod'] + ", " + nlpdata['product consistency'] + ", " + nlpdata['product type'] + ", " + nlpdata['skin concern'] + ", " + nlpdata['coverage type'] + ", " + nlpdata['finish'] + ", " + nlpdata['product preference']

pattern = re.compile(r'(,\s){2,}')

for i in range(len(nlpdata)):
    nlpdata['tags'].iloc[i] = re.sub(pattern, ', ', nlpdata['tags'].iloc[i])

In [24]:
nlpdata = nlpdata.drop(columns=['Unnamed: 0', 'id', 'renzoe tags', 'id.1', 'unnamed: 0', 'unnamed: 0.1'])#, 'product type', 'product consistency', 'pod nonpod', 'product preference', 'skin concern', 'coverage type', 'finish'])
nlpdata = nlpdata.rename(columns = {"body (html)": "body"})

In [22]:
# Output to shopify
# nlpdata.to_csv('nlpresult_2shopify.csv', index=False)

In [23]:
#from google.colab import files
#files.download("nlpresult_2shopify.csv")